In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awt '{print $2}' | xargs -n1 dpkg --purge
!apt-get --remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-11-20 16:45:05--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.20.126
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-11-20 16:45:05--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?dUuPFsMWz00MyDaiclPt9AnqDZZyJ314uEbL79rpY_W4SrynLny7zHF0sd6ozVe42ODYCJnOq3nXLBNLZnkzr-jrUwS2IlgshRppkJ6CnkGgK

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-kd_4ogof
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-kd_4ogof
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=a3f529c88d38cb4481d59053ba02b4dfa50dffd7d25bd1d44e286017d43dacfe
  Stored in directory: /tmp/pip-ephem-wheel-cache-zfolih8l/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# Imagem

In [ ]:
!git clone https://github.com/nothings/stb.git

Cloning into 'stb'...
remote: Enumerating objects: 7844, done.
remote: Counting objects: 100% (573/573), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 7844 (delta 367), reused 514 (delta 332), pack-reused 7271
Receiving objects: 100% (7844/7844), 5.52 MiB | 15.32 MiB/s, done.
Resolving deltas: 100% (5212/5212), done.


In [ ]:
%%cu
#include <stdio.h>
#include <stdint.h>
#include <stdlib.h>

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image/stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image/stb_image_write.h"

#define TILE_W 16
#define TILE_H 16
#define R 2
#define D (R*2+1)
#define S (D*D)
#define BLOCK_W (TILE_W+(R*2))
#define BLOCK_H (TILE_H+(R*2))

__global__ 
void d_filter(int *g_imagem, int *o_imagem, unsigned int altura, unsigned int largura) {
    __shared__ int smen[BLOCK_W*BLOCK_H];
    int x = blockIdx.x * TILE_W + threadIdx.x - R;
    int y = blockIdx.y * TILE_H + threadIdx.y - R;

    x = max(0, x);
    x = min(x, largura - 1);
    y = max(0, y);
    y = min(x, altura - 1);

    unsigned int index1 = y * largura + x;
    unsigned int index2 = threadIdx.y * blockDim.y + threadIdx.x;

    smen[index2] = g_imagem[index];
    __syncthreads();

    if ((threadIdx.x >= R) && (threadId.x < (BLOCK_W - R))
            && (threadIdx.y >= R) && (threadId.y < (BLOCK_H - R)) {
        float soma = 0;
        int dx, dy;
        for (dy = -R; dy <= R; dy++) {
            for (dx = -R; dx <= R; dx++) {
                soma += smen[index2 + (dy * blockDim.x) + dx];
            }
        }
        o_imagem[index1] = soma / S;
    }
}

int main() {
    int alt, lar, bpp;

    uint8_t *rgb_image = stbi_load("imagem", &alt, &lar, &bpp, 3);

    if (rgb_image == NULL) {
        printf("Falha\n");
        exit(1);
    }

    printf("Imagem carregada %dpx %dpx %dpx\n", alt, lar, bpp);
    stbi_image_free(rgb_image);
/*    
    cudaMalloc((void**)&d_g_image, sizeof(int));
    cudaMalloc((void**)&d_o_image, sizeof(int));

    cudaMemcpy(d_g_image, g_image, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_o_image, o_image, sizeof(int), cudaMemcpyHostToDevice);

    d_filter<<<1, 1>>>(d_g_image, d_o_image, alt, lar);

    cudaMemcpy(&resultado, d_o_image, sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_g_image);
    cudaFree(d_o_image);
*/
    return 0;
}



/tmp/tmp5t6zohlp/4b077bef-4219-4e6f-802b-3dbc4f70a3f1.cu:6:10: fatal error: stb_image/stb_image.h: No such file or directory
 #include "stb_image/stb_image.h"
          ^~~~~~~~~~~~~~~~~~~~~~~
compilation terminated.



# ---

In [ ]:
%%cu
#include <stdio.h>
#define N 3
#define SEP ";"
#define M N-1

__global__ void vector(float * coluna) {
    int i = 1;
    int vet[] = {8,7,6};

    for (; i < N; i++) {
        coluna[0] += vet[i];
    }
}

int main() {
    FILE *arq;
    float *coluna, *d_coluna;
    float soma;

    coluna = (float*)malloc(sizeof(float) * M);
    cudaMalloc((void**)&d_coluna, sizeof(float) * M);

    if ((arq = fopen("aluguel_residencial.csv", "r")) == NULL) {
        printf("Falha\n");
        return -1;
    }

    char conteudo[N][100];
    int linha = 0, virg;
    while ((fscanf(arq, " %100[^\n]", conteudo[linha])) != EOF) {
        linha++;
        if (linha >= N) break;
    }

    for (linha = 1; linha < N; linha++) {
        char *token = strtok(conteudo[linha], SEP);
        virg = 1;
        while (token != NULL) {
            if (virg == 7) {
                printf("Token %-30s\tvirgula %d\n", token, virg);
                coluna[linha-1] = atof(token);
            }
            virg++;
            token = strtok(NULL, SEP);
        }
    }
//    cudaMemcpy(d_coluna, coluna, sizeof(float) * M, cudaMemcpyHostToDevice);
    vector<<<1, 1>>>(coluna);
    cudaMemcpy(&soma, coluna, sizeof(float), cudaMemcpyDeviceToHost);

    printf("SOMA %.2f\n", soma);

    cudaFree(d_coluna);
    return 0;
}

# Definitivo

In [ ]:
%%cu
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

#define N 1000
#define SEP ";"

__global__ void vector(float * coluna, float * soma) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) 
        soma[i] += coluna[i];

    for (i = 1; i < N; i++)
        soma[0] += soma[i];
}

int main(){
    int i = 0, virg;
    FILE *arq;
    float soma;
    float *coluna = (float*)malloc(sizeof(float) * N);
    float *d_coluna, *d_soma;
    char conteudo[N][100];

    // Initialize array
    if ((arq = fopen("aluguel_residencial.csv", "r")) == NULL) {
        printf("Falha\n");
        return -1;
    }

    while ((fscanf(arq, " %100[^\n]", conteudo[i])) != EOF) {
        i++;
        if (i >= N)
          break;
    }
    for (i = 1; i < N; i++) {
        char *token = strtok(conteudo[i], SEP);
        virg = 1;
        while (token != NULL) {
            if (virg == 7) {
                coluna[i-1] = atof(token);
            }
            virg++;
            token = strtok(NULL, SEP);
        }
    }

    cudaMalloc((void**)&d_coluna, sizeof(float) * N);
    cudaMalloc((void**)&d_soma, sizeof(float) * N);

    cudaMemcpy(d_coluna, coluna, sizeof(float) * N, cudaMemcpyHostToDevice);
    
    vector<<<1, N>>>(d_coluna, d_soma);

    cudaMemcpy(&soma, d_soma, sizeof(float), cudaMemcpyDeviceToHost);
    printf("SOMA %.2f", soma);

    cudaFree(d_soma);
    cudaFree(d_coluna);
    return 0;
}

/tmp/tmp6txcs_xs/07b63c2e-93cb-425a-9bf7-caf8d27a7886.cu(56): error: expression must have pointer-to-object type

1 error detected in the compilation of "/tmp/tmpxft_0000447a_00000000-8_07b63c2e-93cb-425a-9bf7-caf8d27a7886.cpp1.ii".



In [ ]:
%%cu
#include <stdlib.h>
#include <stdio.h>

//#define N 100000000
#define N 5
#define SEP ";"

__global__ 
void vector_add(float *out, float *coluna) {
    
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) out[i] += coluna[i];

    float aux = out[0];
    for (i = 1; i < N; i++)
        out[0] += out[i];

}

int main(){
    int ndev, i = 0, virg;
    FILE *arq;
    float *coluna;
    float *d_coluna, *d_out; 
    char conteudo[N][100];

    // Allocate memory
    coluna = (float*)malloc(sizeof(float) * N);

    // Initialize array
    if ((arq = fopen("aluguel_residencial.csv", "r")) == NULL) {
        printf("Falha\n");
        return -1;
    }

    while ((fscanf(arq, " %100[^\n]", conteudo[i])) != EOF) {
        i++;
        if (i >= N)
          break;
    }

    for (i = 1; i < N; i++) {
        char *token = strtok(conteudo[i], SEP);
        virg = 1;
        while (token != NULL) {
            if (virg == 7) {
                coluna[i-1] = atof(token);
            }
            virg++;
            token = strtok(NULL, SEP);
        }
    }

    cudaGetDeviceCount(&ndev);
    printf("Devices: %d \n", ndev);

    // Alocate device memory
    cudaMalloc((void**)&d_coluna, sizeof(float) * N);
    cudaMalloc((void**)&d_out, sizeof(float) * N);

    // Transfer data from host to device memory
    cudaMemcpy(d_coluna, coluna, sizeof(float) * N, cudaMemcpyHostToDevice);
    
    float soma;
    // Main function
    vector_add<<<(N+256)/256, 256>>>(d_out, d_coluna);

    cudaMemcpy(&soma, d_out, sizeof(float), cudaMemcpyDeviceToHost);
    printf("SOMA %f", soma);
    return 0;
}

# Oficial

In [ ]:
%%cu
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>
#define N 100

void vector(char * a, int len) {
    for (int i = 0; i < len; i++) {
        if (a[i] == ';') {
            a[i] = ' ';
        }
    }
}

int main() {
    FILE *arq;
    char conteudo[N][N];
    char *d_conteudo;
    int i = 0;

    if ((arq = fopen("aluguel_residencial.csv", "r")) == NULL) {
        printf("Falha!\n");
        return;
    }

    while ((fscanf(arq, "%s", conteudo[i])) != EOF) {
        printf("[%d]: %s\n", i, conteudo[i]);
        i++;
    }
    
    for (int i = 0; i < 2; i++) {
//        fscanf(arq, "%s", conteudo);
//        cudaMalloc((void**)&d_conteudo, sizeof(char) * N);
//        cudaMemcpy(&d_conteudo, conteudo, sizeof(char) * N, cudaMemcpyHostToDevice);
//        vector(conteudo, strlen(conteudo));
//        cudaMemcpy(&conteudo, d_conteudo, sizeof(char) * N, cudaMemcpyDeviceToHost);
        printf("[%d]: %s\n", i, conteudo[i]);
    }
    return 0;
}

//void t() {
//    int *x, *d_x;
//    int resultado;
//    x = (int *)malloc(sizeof(int));
//    cudaMalloc((void**)&d_x, sizeof(int));
//    cudaMemcpy(&d_x, x, sizeof(int), cudaMemcpyHostToDevice);
//    vector<<<1,1>>>(d_x, 12);
//    cudaMemcpy(&x, d_x, sizeof(int), cudaMemcpyDeviceToHost);
//    printf("x = %d\n", x);
//}

# #

In [ ]:
%%cu
#include <stdio.h>
#include <string.h>
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>

#define N 3

__global__
void vector(int * csum, float * mean, char [N][N] csv) {
    int coluna = 0, j;
    char * token = strtok(csv, ",");

    while (token != NULL) {
        token = strtok(NULL, ",");
        coluna++;
        // Nome de um terminal do csv
        if (strcmp(token, "BEM2021")) {
            token = strtok(NULL, ",");
            // 4 = distância até a coluna numérica
            for (j = 0; j < 4; j++) {
                char * tok = strtok(NULL, ",");
                *csum += atoi(tok);
            }
            coluna = 0;
            break; // continue;
        }
    }
    *mean = 5.8;
}

int main() {
    FILE *arq;
    char *d_conteudo;
    char conteudo[N][N], ch;
    int i = 0, j = 0;
    int *csum, *d_csum;
    float *mean, *d_mean;

    csum = (int*)malloc(sizeof(int));
    mean = (float*)malloc(sizeof(float));
    csv = (char*)malloc(sizeof(char));

    cudaMalloc((void**)&d_csum, sizeof(int));
    cudaMalloc((void**)&d_mean, sizeof(float));
    cudaMalloc((void**)&csv, sizeof(char) * N * N);

    // Leitura do arquivo
    if ((arq = fopen("aluguel_residencial.csv", "r")) == NULL) {
        printf("Falha\n");
        return -1;
    }

    while ((ch = fgetc(arq)) != EOF) {
        if (ch == '\0' || ch == '\n') {
            i++;
            j = 0;
        } else  {
            conteudo[i][j++] = ch;
        }
    }

    cudaMemcpy(&d_csum, csum, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(&d_mean, mean, sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(&d_conteudo, conteudo, sizeof(char) * N * N, cudaMemcpyHostToDevice);

    vector<<<1, 1>>>(d_csum, d_mean, d_conteudo);

    cudaMemcpy(&csum, d_csum, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&mean, d_mean, sizeof(float), cudaMemcpyDeviceToHost);
    printf("Csum = %d\tMean = %lf\n", csum, mean);

    cudaFree(d_csum);
    cudaFree(d_mean);
    return 0;
}

# Nova Solução

In [ ]:
%%cu
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>

#define N 76
#define SEP ";"
//#define BAIRRO "Copacabana"

__global__ void vector(float *, float *, float *);

__global__
void vector(float * coluna, float * media, float * soma) {
    int i = threadIdx.x;
    if (i < N) 
      *soma = soma[i] + coluna[i];
    
    *media = 55.6f;
}

int main() {
    FILE *arq;
    int i = 0, virg;
    float *coluna = (float*)malloc(sizeof(float));
    float *media = (float*)malloc(sizeof(float));
    float *soma = (float*)malloc(sizeof(float));
    char conteudo[N][1000];

    float *d_coluna, *d_media, *d_soma;

    cudaMalloc((float**)&d_coluna, sizeof(float) * N);
    cudaMalloc((float**)&d_media, sizeof(float));
    cudaMalloc((float**)&d_soma, sizeof(float));
    
    if ((arq = fopen("aluguel_residencial.csv", "r")) == NULL) {
        printf("Falha\n");
        return -1;
    }

    while ((fscanf(arq, " %100[^\n]", conteudo[i])) != EOF) {
        i++;
        if (i >= N)
          break;
    }

    for (i = 1; i < N; i++) {
        char *token = strtok(conteudo[i], SEP);
        virg = 1;
        while (token != NULL) {
            if (virg == 7) {
                coluna[i-1] = atof(token);
            }
            virg++;
            token = strtok(NULL, SEP);
        }
    }
    
    cudaMemcpy(&d_coluna, coluna, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(&d_media, media, sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_soma, soma, sizeof(float), cudaMemcpyHostToDevice);

    vector<<<1, 3>>>(d_coluna, d_media, d_soma);

    cudaMemcpy(soma, d_soma, sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(media, d_media, sizeof(float), cudaMemcpyDeviceToHost);

    float sum = 0;
    for (i = 0; i < N; i++) {
        //printf("%.2f\n", soma[i]);
        sum += soma[i];
    }

    printf("Sum = %.2lf\n", sum);

    free(coluna);
    free(media);
    free(soma);
    cudaFree(d_coluna);
    cudaFree(d_media);
    cudaFree(d_soma);

    return 0;
}

In [ ]:
filtro = aluguel['Bairro'] == 'Olaria'
filtro = aluguel[filtro]
# filtro.Valor.mean()
aluguel.head(76).Valor.mean()

In [ ]:
%%cu
// Testando a solução de enviar apenas uma coluna para o kernel
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <ctype.h>
#define N 1000

int main() {
    FILE *arq;
    int i = 0;
//    float kernel[5];
    char ch[N][100];

    // Leitura do arquivo
    if ((arq = fopen("aluguel_residencial.csv", "r")) == NULL) {
        printf("Falha\n");
        return -1;
    }

    while ((fscanf(arq, " %100[^\n]", ch[i])) != EOF) {
        printf("linha %d: %s\t%s\n", i++, ch[i]);
        if (i >= 5)
          break;
    }
    
    int aux = 0;
    float *valores = (float*)malloc(sizeof(float));
    for (i = 1; i < 5; i++) {
        char *token = strtok(ch[i], ";");
        int vgl = 1;
        while (token != NULL) {
            if (vgl == 7) {
                valores[aux++] = atof(token);
                printf("Token %.2f - %s\n", valores[aux-1], token);
            }
            vgl++;
            token = strtok(NULL, ";");
        }
    }
    return 0;
}

In [ ]:
import pandas as pd
aluguel = pd.read_csv('aluguel_residencial.csv', sep=";")
# filtro = aluguel['Bairro'] == 'Copacabana'
# filtro = aluguel[filtro]
# filtro
aluguel.head()

# Outro

In [ ]:
%%cu
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>
#define N 100

void vector(char[*][] a) {
    for (int i = 0; i < 2; i++) {
        for (int j = 0; j < strlen(a[i]); j++) {
            if (a[i][j] == ';') {
                a[i][j] = ' ';
            }
        }
    }
}

int main() {
    FILE *arq;
    char conteudo[N][N];
    int i = 0;

    if ((arq = fopen("aluguel_residencial.csv", "r")) == NULL) {
        printf("Falha!\n");
        return;
    }

    while ((fscanf(arq, "%s", conteudo[i])) != EOF) {
        printf("[%d]: %s\n", i, conteudo[i]);
        i++;
    }
    vector(conteudo);

    for (int i = 0; i < 2; i++) {
        printf("[%d]: %s\n", i, conteudo[i]);
    }
    return 0;
}

In [ ]:
%%cu
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ 
void add(int x, int y, int * n) {
    *n = x + y;
}

int main() {
    int h_c;
    int *n;
    cudaMalloc((void**)&n, sizeof(int));
    add << <1, 1>> > (1, 4, n);
    cudaMemcpy(&h_c, n, sizeof(int), cudaMemcpyDeviceToHost);
    printf("1 + 4 = %d\n", h_c);
    cudaFree(n);
    return 0;
}

# Código

In [ ]:
%%cu
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ 
void add(int x, int y, int * n) {
    *n = x + y;
}

int main() {
    int h_c;
    int *n;
    cudaMalloc((void**)&n, sizeof(int));
    add <<<1, 8>>>(3, 4, n);
    cudaMemcpy(&h_c, n, sizeof(int), cudaMemcpyDeviceToHost);
    printf("1 + 4 = %d\n", h_c);
    cudaFree(n);
    return 0;
}

void lerDataSet() {
    // TODO
}

In [ ]:
%%cu
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>
#define N 10

___global__ void vector(int * a, int * b, int * c) {
    int i = threadIdx.x;
    if (i < N)
        c[i] = a[i] + b[i];
}

int main() {
    int *a, *b, *c;        // ponteiros do host
    int *d_a, *d_b, *d_c;  // ponteiros do device

    a = (int*) malloc(sizeof(int) * N);
    b = (int*) malloc(sizeof(int) * N);
    c = (int*) malloc(sizeof(int) * N);

    for (int i = 0; i < N; i++) {
        a[i] = i;
        b[i] = i - 3;
        c[i] = 0;
        printf("%d\n", a[i]);
    }

 //   cudaMalloc((void**)&d_a, sizeof(int) * N);
   // cudaMalloc((void**)&d_b, sizeof(int) * N);
    //cudaMalloc((void**)&d_c, sizeof(int) * N);

    // Copia os dados da GPU para a CPU
    // destino, origem, bytes, modo
   // cudaMemcpy(d_a, a, sizeof(int) * N, cudaMempyHostToDevice);
    //cudaMemcpy(d_b, b, sizeof(int) * N, cudaMemcpyHostToDevice);

    // Chamada a criação do vetor (threads) <<<núcleos, threads>>>
    vector<<<1, N>>>(a, b, c);

    cudaMemcpy(d_c, c, sizeof(int) * N, cudaMemcpyDeviceToHost);

    for (int j = 0; j < N; j++) 
        printf("c[%d] = %d\n", j, c[j]);

    cudaFree(a);
    cudaFree(b);
    cudaFree(c);

    return 0;
}

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <iostream>

__global__
void vector(int * arq) {
    //*x = 98;
    int i = threadIdx.x;
}

int main() {
    FILE *arq;
    int palavra[100];
    int *x;
    int h_x;
    
    if ((arq = fopen("aluguel_residencial.csv", "r")) == NULL) {
        printf("Falha\n");
        return;
    }
    // Alocar memória
    // passar pro cuda

    while (fscanf(arq, "%s", palavra) != EOF) {
        printf("%s\n", palavra);
    }

    cudaMalloc((void**)&x, sizeof(int));
    vector<<<1,10>>>(palavra);

    // Retorna o resultado para variavel da main
    cudaMemcpy(&h_x, palavra, sizeof(int), cudaMemcpyDeviceToHost);
    printf("\n\nSaida %c\n", h_x);
    return 0;
}

# Exemplos

In [ ]:
%%cu
#include <stdlib.h>
#include <stdio.h>
#define N 100000000

void vector_add(float *out, float *a, float *b, int n) {
    for(int i = 0; i < n; i++){
        out[i] = a[i] + b[i];
    }

    for (int i = 0; i < 10; i++) {
        printf("out[%d] = %.2f\n", i, out[i]);
    }
}

int main(){
    float *a, *b, *out; 

    // Allocate memory
    a   = (float*)malloc(sizeof(float) * N);
    b   = (float*)malloc(sizeof(float) * N);
    out = (float*)malloc(sizeof(float) * N);

    // Initialize array
    for(int i = 0; i < N; i++){
        a[i] = 1.0f; 
        b[i] = 2.0f;
    }

    // Main function
    vector_add(out, a, b, N);
}

In [ ]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#define N 100000000

__global__ 
void vector_add(float *out, float *a, float *b, int n) {
    
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) out[i] = a[i] + b[i];

}

void printDevProp(cudaDeviceProp devProp)
{
    printf("Major revision number:         %d\n",  devProp.major);
    printf("Minor revision number:         %d\n",  devProp.minor);
    printf("Name:                          %s\n",  devProp.name);
    printf("Total global memory:           %lu\n",  devProp.totalGlobalMem);
    printf("Total shared memory per block: %lu\n",  devProp.sharedMemPerBlock);
    printf("Total registers per block:     %d\n",  devProp.regsPerBlock);
    printf("Warp size:                     %d\n",  devProp.warpSize);
    printf("Maximum memory pitch:          %lu\n",  devProp.memPitch);
    printf("Maximum threads per block:     %d\n",  devProp.maxThreadsPerBlock);
    for (int i = 0; i < 3; ++i)
        printf("Maximum dimension %d of block:  %d\n", i, devProp.maxThreadsDim[i]);

    for (int i = 0; i < 3; ++i)
        printf("Maximum dimension %d of grid:   %d\n", i, devProp.maxGridSize[i]);

    printf("Clock rate:                    %d\n",  devProp.clockRate);
    printf("Total constant memory:         %lu\n",  devProp.totalConstMem);
    printf("Texture alignment:             %lu\n",  devProp.textureAlignment);
    printf("Concurrent copy and execution: %s\n",  (devProp.deviceOverlap ? "Yes" : "No"));
    printf("Number of multiprocessors:     %d\n",  devProp.multiProcessorCount);
    printf("Cuda Cores:                    %d\n", devProp.multiProcessorCount * 64); // ver 7.5
    printf("Kernel execution timeout:      %s\n",  (devProp.kernelExecTimeoutEnabled ? "Yes" : "No"));
    return;
}

int main(){
    int ndev;

    float *a, *b;
    float *d_a, *d_b, *d_out; 

    // Allocate memory
    a = (float*)malloc(sizeof(float) * N);
    b = (float*)malloc(sizeof(float) * N);

    // Initialize array
    for(int i = 0; i < N; i++){
        a[i] = 1.0f; b[i] = 2.0f;
    }

    cudaGetDeviceCount(&ndev);
    printf("Devices: %d \n", ndev);

    cudaDeviceProp pdev;
    for (int i = 0; i < ndev; i++) {
        cudaGetDeviceProperties(&pdev, i);
        printDevProp(pdev);
    }

    // Alocate device memory
    cudaMalloc((void**)&d_a, sizeof(float) * N);
    cudaMalloc((void**)&d_b, sizeof(float) * N);
    cudaMalloc((void**)&d_out, sizeof(float) * N);

    // Transfer data from host to device memory
    cudaMemcpy(d_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);
    
    // Main function
    vector_add<<<(N+256)/256, 256>>>(d_out, d_a, d_b, N);
}

In [ ]:
%%cu
#include <stdlib.h>
#include <stdio.h>

void mult_matrix(float *m_a, float*m_b, float *m_c, int n)
{
    for (int i=0; i<n; i++)
        for ( int j=0; j<n; j++) 
        {
            m_c[(i*n)+j] = 0.0;
            for ( int k=0; k<n; k++)
                m_c[(i*n)+j] = m_c[(i*n)+j] + m_a[(i*n)+k] * m_b[(k*n)+j];
        }
}

__global__ 
void mult_matrix_kernel(float *m_a, float *m_b, float *m_c, int n) {
    
    int i = blockIdx.y*blockDim.y+threadIdx.y;
    int j = blockIdx.x*blockDim.x+threadIdx.x;

    if ((i < n) && (j < n)) {
        float c = 0;
        for (int k = 0; k < n; ++k) {
            c += m_a[(i*n)+k] * m_b[(k*n)+j];
        }
    
        m_c[(i*n)+j] = c;
    }
}
    
void print_matrix(float *m, int n, char l){

    if (n > 10) return;

    printf("Matrix: %c \n", l);

    for(int i=0; i<n; i++)
        for(int j=0; j<n; j++)
            printf("%f \n", m[(i*n)+j]);

    printf("\n\n");
}


int main(int argc, char *argv[]) {

    int n, t;
    float *m_a, *m_b, *m_c; //matrizes "linearizadas"
    float *d_m_a, *d_m_b, *d_m_c;

    if(argc < 3) return -1;
    n = atoi(argv[1]);
    t = atoi(argv[2]);

    m_a = (float*)malloc(n * n * sizeof(float));
    m_b = (float*)malloc(n * n * sizeof(float));
    m_c = (float*)malloc(n * n * sizeof(float));

    for(int i=0; i<n; i++)
        for(int j=0; j<n; j++)
        {
            m_a[(i*n)+j] = i+1;
            m_b[(i*n)+j] = (i+1) + (j+1);
            m_c[(i*n)+j] = 0.0;
        }

    print_matrix(m_a, n, 'A');
    print_matrix(m_b, n, 'B');

    if(t == 0) 
    {
        mult_matrix(m_a, m_b, m_c, n);
    }
    else 
    {
        cudaMalloc((void**)&d_m_a, sizeof(float) * n * n);
        cudaMalloc((void**)&d_m_b, sizeof(float) * n * n);
        cudaMalloc((void**)&d_m_c, sizeof(float) * n * n);
        
        cudaMemcpy(d_m_a, m_a, sizeof(float) * n * n, cudaMemcpyHostToDevice);
        cudaMemcpy(d_m_b, m_b, sizeof(float) * n * n, cudaMemcpyHostToDevice);
    
        dim3 blockDim(16, 16);
        dim3 gridDim(ceil(((float)n) / blockDim.x), ceil(((float)n) / blockDim.y));

        mult_matrix_kernel<<<gridDim, blockDim>>>(d_m_a, d_m_b, d_m_c, n);
        cudaMemcpy(m_c, d_m_c, sizeof(float) * n * n, cudaMemcpyDeviceToHost);
    }
    
    print_matrix(m_c, n, 'C');

    return 0;
}